In [1]:
import torch
import math
import matplotlib.pyplot as plt
import numpy as np


In [2]:
# 迭代次数
EPOCH = 1500
# 学习率
MLP_LR = 0.01
left, right = -2,2
PI = math.pi
NUM=50.0

In [3]:
type(NUM)

float

In [4]:
x = torch.tensor([(num * PI) for num in np.arange(left, right, (right - left) / NUM)], dtype=torch.float)
print(x)
y = torch.sin(x)
print(y)

tensor([-6.2832e+00, -6.0319e+00, -5.7805e+00, -5.5292e+00, -5.2779e+00,
        -5.0265e+00, -4.7752e+00, -4.5239e+00, -4.2726e+00, -4.0212e+00,
        -3.7699e+00, -3.5186e+00, -3.2673e+00, -3.0159e+00, -2.7646e+00,
        -2.5133e+00, -2.2619e+00, -2.0106e+00, -1.7593e+00, -1.5080e+00,
        -1.2566e+00, -1.0053e+00, -7.5398e-01, -5.0265e-01, -2.5133e-01,
         5.5806e-15,  2.5133e-01,  5.0265e-01,  7.5398e-01,  1.0053e+00,
         1.2566e+00,  1.5080e+00,  1.7593e+00,  2.0106e+00,  2.2619e+00,
         2.5133e+00,  2.7646e+00,  3.0159e+00,  3.2673e+00,  3.5186e+00,
         3.7699e+00,  4.0212e+00,  4.2726e+00,  4.5239e+00,  4.7752e+00,
         5.0265e+00,  5.2779e+00,  5.5292e+00,  5.7805e+00,  6.0319e+00])
tensor([-1.7485e-07,  2.4869e-01,  4.8175e-01,  6.8455e-01,  8.4433e-01,
         9.5106e-01,  9.9803e-01,  9.8229e-01,  9.0483e-01,  7.7051e-01,
         5.8779e-01,  3.6812e-01,  1.2533e-01, -1.2533e-01, -3.6812e-01,
        -5.8779e-01, -7.7051e-01, -9.0483e-01, -9.

In [ ]:
plt.plot(x,y,color='red')
plt.show()

: 

: 

In [5]:
EPOCH=1500
MLP_LR=0.01
left,right=-2,2
PI=math.pi
NUM=50

# x,y 是普通sinx 的torch tensor
x =torch.tensor([(num * PI)  for num in numpy.arange(left, right,(right-left)/NUM)],dtype=torch.float32)
print(x)
y = torch.sin(x)
plt.plot(x,y,color='red')
plt.show()

NameError: name 'numpy' is not defined